# Neural Network vs LightGBM comparison
### Damjan Strbac

## Neural network

In [1]:
import torch as T
import torch.nn as nn

In [7]:
#Neural network for RCV1 dataset
#dataset: https://scikit-learn.org/0.18/datasets/rcv1.html
class RCV1NeuralNetwork(nn.Module):
    def __init__(self):
        super(RCV1NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(47236, 1024)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.2)
        self.layer2 = nn.Linear(1024, 256)
        self.act2 = nn.ReLU()
        self.drop2 = nn.Dropout(p=0.2)
        self.output = nn.Linear(256, 103)
        self.softmax = nn.LogSoftmax(dim=1) 

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.drop1(x)
        x = self.act2(self.layer2(x))
        x = self.drop2(x)
        x = self.output(x)
        x = self.softmax(x)  # Apply softmax activation
        return x

In [2]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1(subset='train', random_state=42, download_if_missing=True)

X = rcv1.data
y = rcv1.target

In [ ]:
print(X.shape)
print(y.shape)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)


# Convert data and labels to PyTorch tensors
X_train_tensor = T.tensor(X_train.toarray()).float()
X_test_tensor = T.tensor(X_test.toarray()).float()
y_train_tensor = T.tensor(y_train.toarray()).float()
y_test_tensor = T.tensor(y_test.toarray()).float()


In [4]:
from torch.utils.data import DataLoader, TensorDataset

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [5]:
device = T.device("cuda:0") if T.cuda.is_available() else T.device("cpu")
print(device)

cpu


In [8]:
net = RCV1NeuralNetwork().to(device)
optimizer = T.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [9]:

from torcheval.metrics.functional import multiclass_precision, multiclass_recall

def calculate_precision(y_true, y_pred):
    y_pred_classes = y_pred.argmax(dim=1)  # Get predicted class
    y_true_classes = y_true.argmax(dim=1)  # Get true class
    return multiclass_precision(y_pred_classes, y_true_classes, num_classes=103)  # Calculate multiclass precision

# Function to calculate recall for the neural network
def calculate_recall(y_true, y_pred):
    y_pred_classes = y_pred.argmax(dim=1)  # Get predicted class
    y_true_classes = y_true.argmax(dim=1)  # Get true class
    return multiclass_recall(y_pred_classes, y_true_classes, num_classes=103)  # Calculate multiclass recall

In [10]:
EPOCHS = 10
best_precision = 0.0
best_recall = 0.0

best_state = None
best_optimizer_state = None

patience = 2
counter = 0

In [ ]:
for epoch in range(EPOCHS):
    net.train()  # Set the model to training mode
    total_loss = 0.0
    total_precision = 0.0
    
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        preds = net(x)
        loss = loss_fn(preds, y.argmax(dim=1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * x.size(0)
        total_precision += calculate_precision(y, preds).item() * x.size(0)
    
    avg_loss = total_loss / len(train_loader.dataset)
    avg_precision = total_precision / len(train_loader.dataset)
    
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {avg_loss:.4f}, Precision: {avg_precision:.4f}")
    
    # Evaluation on the test set
    net.eval()  # Set the model to evaluation mode
    total_precision = 0.0
    total_recall = 0.0
    
    with T.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            preds = net(x)
            total_precision += calculate_precision(y, preds).item() * x.size(0)
            total_recall += calculate_recall(y, preds).item() * x.size(0)
    
    avg_precision = total_precision / len(test_loader.dataset)
    avg_recall = total_recall / len(test_loader.dataset)
    print(f"Test Precision: {avg_precision:.4f}, Test Recall: {avg_recall:.4f}")
    
    # Check if test accuracy improved
    if avg_precision > best_precision and avg_recall > best_recall:
        best_precision = avg_precision
        best_recall = avg_recall
        best_state = net.state_dict()
        best_optimizer_state = optimizer.state_dict()  # Save optimizer state
        T.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            }, "best_model_checkpoint.pth")
        counter = 0  # Reset counter if there's improvement
    else:
        counter += 1  # Increment counter if no improvement
    
    # Check if early stopping criteria met
    if counter >= patience:
        print(f"Early stopping at epoch {epoch + 1} due to no improvement in test precision.")
        break

In [ ]:
if best_state is not None:
    checkpoint = T.load("best_model_checkpoint.pth")
    net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print("Model restored to the state with the best test precision.")
    
    # Save the best model and optimizer state
    T.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "best_model_checkpoint.pth")

# Evaluate the restored model on the test set
net.eval()
total_precision = 0.0
total_recall = 0.0
with T.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        preds = net(x)
        total_precision += calculate_precision(y, preds).item() * x.size(0)
        total_recall += calculate_recall(y, preds).item() * x.size(0)

nn_final_precision = total_precision / len(test_loader.dataset)
nn_final_recall = total_recall / len(test_loader.dataset)
print("Final precision: ", nn_final_precision)
print("Final reacal: ", nn_final_recall)


## LightGBM

In [15]:
import lightgbm as lgb
import optuna as opt
import numpy as np
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [16]:
X = rcv1.data[:10000]
y = rcv1.target[:10000].toarray()

In [17]:

X_train, X_val,y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

In [ ]:
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)
y_test = np.argmax(y_test, axis=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [19]:
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [21]:
def objective(trial):
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1e-1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = np.rint(y_pred)
    precision = precision_score(y_test, y_pred, average='micro')
    return precision

In [ ]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

In [23]:
best_params = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "num_class": 103,
    "lambda_l1": study.best_params["lambda_l1"],
    "lambda_l2": study.best_params["lambda_l2"],
    "learning_rate": study.best_params["learning_rate"],
    "num_leaves": study.best_params["num_leaves"],
    "feature_fraction": study.best_params["feature_fraction"],
    "bagging_fraction": study.best_params["bagging_fraction"],
    "bagging_freq": study.best_params["bagging_freq"],
    "min_child_samples": study.best_params["min_child_samples"],
}

In [ ]:
final_model = lgb.train(best_params, lgb.Dataset(X_train, label=y_train))
y_pred = final_model.predict(X_test).argmax(axis=1)
lightgbm_final_precision = precision_score(y_test, y_pred, average='micro')
lightgbm_final_recall = recall_score(y_test, y_pred, average='micro')
print("Precision: ", lightgbm_final_precision)
print("Recall: ", lightgbm_final_recall)

# Conclusion

In [ ]:
if lightgbm_final_precision > nn_final_precision:
    print("LightGBM has better precision")

else:
    print("Neural network has better precision")

if lightgbm_final_recall > nn_final_recall:
    print("LightGBM has better recall")
else:
    print("Neural network has better recall")
    